### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [1]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [8]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    # Implement your code here
    reviews = reviews.sort_values(by=[order_by])
    training_df = reviews.iloc[:training_size, :]
    validation_df = reviews.iloc[training_size:(training_size+testing_size), :]
    return training_df, validation_df

In [9]:
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [18]:
val_df.head()

,user_id,movie_id,rating,timestamp,date
71381,5551,841046,6,1374271960,2013-07-19 15:12:40
82410,6410,1245492,8,1374274664,2013-07-19 15:57:44
43388,3251,317705,5,1374274858,2013-07-19 16:00:58
39748,2894,1602613,3,1374275761,2013-07-19 16:16:01
36976,2625,169547,10,1374277805,2013-07-19 16:50:05


In [11]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-07-19', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-08-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 300 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.

In [12]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [24]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, 
                              latent_features=15, 
                              learning_rate=0.005, 
                              iters=300)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 9.383437
2 		 5.110159
3 		 3.743459
4 		 2.944095
5 		 2.397295
6 		 1.993449
7 		 1.681278
8 		 1.432639
9 		 1.230464
10 		 1.063625
11 		 0.924434
12 		 0.807335
13 		 0.708174
14 		 0.623759
15 		 0.551579
16 		 0.489616
17 		 0.436226
18 		 0.390052
19 		 0.349973
20 		 0.315054
21 		 0.284520
22 		 0.257723
23 		 0.234122
24 		 0.213264
25 		 0.194767
26 		 0.178311
27 		 0.163625
28 		 0.150479
29 		 0.138677
30 		 0.128053
31 		 0.118465
32 		 0.109788
33 		 0.101919
34 		 0.094766
35 		 0.088250
36 		 0.082301
37 		 0.076861
38 		 0.071875
39 		 0.067298
40 		 0.063089
41 		 0.059212
42 		 0.055635
43 		 0.052329
44 		 0.049271
45 		 0.046436
46 		 0.043805
47 		 0.041361
48 		 0.039087
49 		 0.036969
50 		 0.034994
51 		 0.033150
52 		 0.031427
53 		 0.029815
54 		 0.028305
55 		 0.026891
56 		 0.025564
57 		 0.024318
58 		 0.023147
59 		 0.022047
60 		 0.021011
61 		 0.020035
62 		 0.019116
63 		 0.018249
64 		 0

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [33]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Create series of users and movies in the right order
    user_ids_series = np.array(train_data_df.index)
    movie_ids_series = np.array(train_data_df.columns)

    # User row and Movie Column
    user_row = np.where(user_ids_series == user_id)[0][0]
    movie_col = np.where(movie_ids_series == movie_id)[0][0]
    # Take dot product of that row and column in U and V to make prediction
    pred = np.dot(user_matrix[user_row, :], movie_matrix[:, movie_col])
    return pred

In [34]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 2625, 169547)
pred_val

9.7719806361559147

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [27]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    '''
    movie_name = movies[movies['movie_id'] == movie_id]['movie']# Implement your code here

    print("For user {} we predict a {} rating for the movie {}.".format(user_id, round(prediction, 2), str(movie_name)))

In [28]:
# Test your function the the results of the previous function
print_prediction_summary(2625, 169547, pred_val)

For user 2625 we predict a 9.77 rating for the movie 10467    American Beauty (1999)
Name: movie, dtype: object.


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [20]:
val_df.head()

,user_id,movie_id,rating,timestamp,date
71381,5551,841046,6,1374271960,2013-07-19 15:12:40
82410,6410,1245492,8,1374274664,2013-07-19 15:57:44
43388,3251,317705,5,1374274858,2013-07-19 16:00:58
39748,2894,1602613,3,1374275761,2013-07-19 16:16:01
36976,2625,169547,10,1374277805,2013-07-19 16:50:05


In [36]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    # Implement your code here
    val_users = np.array(val_df['user_id'])
    val_movies = np.array(val_df['movie_id'])
    val_ratings = np.array(val_df['rating']) 
    # Fit FunkSVD with the specified hyper parameters to the training data
    for i in range(num_preds):
        user_id = val_users[i]
        movie_id = val_movies[i]
        movie_name = movies[movies['movie_id'] == movie_id]['movie']
        act_rating = val_ratings[i]
        try:
            pred = predict_rating(user_mat, movie_mat, user_id, movie_id)
            print("For user {} we predict a {} rating for the movie {} but the actual rating is {}.".format(user_id, round(pred, 2), str(movie_name), act_rating))
        except IndexError:
            continue
# Perform the predicted vs. actual for the first 10 rows.  How does it look?
validation_comparison(val_df, 10)

For user 6410 we predict a 4.21 rating for the movie 17590    This Is the End (2013)
Name: movie, dtype: object but the actual rating is 8.
For user 2894 we predict a 3.42 rating for the movie 19473    Only God Forgives (2013)
Name: movie, dtype: object but the actual rating is 3.
For user 2625 we predict a 9.77 rating for the movie 10467    American Beauty (1999)
Name: movie, dtype: object but the actual rating is 10.
For user 5490 we predict a 6.26 rating for the movie 15261    Man of Steel (2013)
Name: movie, dtype: object but the actual rating is 6.
